<a href="https://colab.research.google.com/github/gfdiazc/bender-app/blob/main/Aplicacio%CC%81n_para_ScreenShoot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# First Cell: Install Dependencies
!pip install selenium -q
!apt-get update -q
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 25.1 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,633 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InR

In [3]:
# Second Cell: Import Libraries
import os
import time
import zipfile
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, FileLink
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Third Cell: Configure Chrome Options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

In [5]:
# Fourth Cell: Constants and Utility Functions
# Constants for resolutions
RESOLUTIONS = {
    "desktop": (1920, 1080),
    "tablet": (768, 1024),
    "mobile": (375, 812)
}

# Ensure output directories exist
def create_output_directories(base_dir="/content/drive/My Drive/screenshots"):
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    for device in RESOLUTIONS.keys():
        os.makedirs(os.path.join(base_dir, device), exist_ok=True)

# Validate URLs
def validate_url(url):
    if not url.startswith("http://") and not url.startswith("https://"):
        return False
    return True

In [6]:
# Fifth Cell: Screenshot and Processing Functions
# Capture screenshot
def capture_screenshot(driver, url, output_path, width, height):
    driver.set_window_size(width, height)
    driver.get(url)

    try:
        # Wait for the page to load fully
        time.sleep(3)

        # Handle cookies pop-up if present
        try:
            cookie_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Aceptar') or contains(text(), 'Aceptar todas') or contains(text(), 'Agree') or contains(text(), 'Accept')]"))
            )
            cookie_button.click()
            time.sleep(2)  # Wait for the pop-up to disappear
        except Exception:
            pass  # No cookie pop-up found, continue

        # Adjust window size for full page
        total_height = driver.execute_script("return document.body.scrollHeight")
        driver.set_window_size(width, total_height)
        driver.save_screenshot(output_path)
    except Exception as e:
        print(f"Error capturing screenshot for {url}: {e}")

# Process URLs
def process_urls(urls, base_dir="/content/drive/My Drive/screenshots"):
    report = []
    driver = webdriver.Chrome(options=chrome_options)

    for url in urls:
        if not validate_url(url):
            report.append({"URL": url, "Status": "Invalid URL", "Error": "Invalid URL format"})
            continue

        for device, (width, height) in RESOLUTIONS.items():
            try:
                output_path = os.path.join(base_dir, device, f"{url.replace('http://', '').replace('https://', '').replace('/', '_').replace('?', '_').replace('&', '_')}.png")
                capture_screenshot(driver, url, output_path, width, height)
                report.append({"URL": url, "Device": device, "Status": "Success", "Error": None})
            except Exception as e:
                report.append({"URL": url, "Device": device, "Status": "Failed", "Error": str(e)})

    driver.quit()
    return report

In [7]:
# Sixth Cell: Save Report and Create ZIP
# Save report as CSV
def save_report(report, output_file="/content/drive/My Drive/screenshots/report.csv"):
    df = pd.DataFrame(report)
    df.to_csv(output_file, index=False)
    return output_file

# Create ZIP archive of screenshots
def create_zip_archive(base_dir="/content/drive/My Drive/screenshots", zip_name="/content/drive/My Drive/screenshots/screenshots.zip"):
    with zipfile.ZipFile(zip_name, "w") as zipf:
        for root, dirs, files in os.walk(base_dir):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, base_dir))
    return zip_name

In [8]:
# Seventh Cell: Main Execution
# Example usage in Colab
def main():
    # Example: Load URLs from a CSV or manually define them
    urls = [
        "https://www.pepephone.com/",

    ]

    # Create output directories
    create_output_directories()

    # Process URLs
    report = process_urls(urls)

    # Save report
    report_file = save_report(report)
    display(FileLink(report_file))

    # Create ZIP archive
    zip_file = create_zip_archive()
    display(FileLink(zip_file))

if __name__ == "__main__":
    main()

/content/drive/My Drive/screenshots/report.csv

/content/drive/My Drive/screenshots/screenshots.zip